<a href="https://colab.research.google.com/github/Stephanieessmak/database/blob/main/Mumzworld_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe

from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from itertools import repeat
import warnings
from google.colab import drive
drive.mount('/content/drive')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

Mounted at /content/drive


In [ ]:
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
json = '/content/drive/MyDrive/Essmak Data Work/Requirements/Credentials.json'
credentials = Credentials.from_service_account_file(json, scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth() 
drive = GoogleDrive(gauth) 

# data = pd.read_csv('/content/drive/MyDrive/Essmak Data Work/CRM /order-data_2023-03-11.csv')
df = gc.open_by_url("https://docs.google.com/spreadsheets/d/1durdZwaOeXVA1AeYYwUTeg2Y6BPgyaWJhzWhl8RAn4E/edit#gid=0")
AED = df.worksheet('AED To USD')
AED=pd.DataFrame(AED.get_all_records())                                                                     

df = gc.open_by_url("https://docs.google.com/spreadsheets/d/1jdCfmvWAuRnyPVfR7lx38-jXKNeSOno-vmKnU1fjhk4/edit#gid=497917819")
sku = df.worksheet('Combined SKU Details')
sku=pd.DataFrame(sku.get_all_records()) 


# reading cost sheet data
cp = gc.open_by_url('https://docs.google.com/spreadsheets/d/18wCjME5w01pB3TTIXcQaq1KnrtiTORrqkUHiNIzycZk/edit#gid=227041125')
cp = cp.worksheet('Cost Sheet')
cp =pd.DataFrame(cp.get_all_records())
ff = gc.open_by_url('https://docs.google.com/spreadsheets/d/18wCjME5w01pB3TTIXcQaq1KnrtiTORrqkUHiNIzycZk/edit#gid=227041125')
ff = ff.worksheet('Franchise Fee') 
ff =pd.DataFrame(ff.get_all_records())

df2 = pd.read_excel("/content/drive/MyDrive/Essmak Data Work/MumzWorld/Mumzworld.xlsx", sheet_name='Essmak')


In [ ]:
ff=ff.loc[ff['Platform']=='Mumzworld']

In [ ]:
df2['Date'] = pd.to_datetime(df2['Date'])
df2['day'] = df2['Date'].dt.day 
df2['month'] = df2['Date'].dt.month
df2['year'] = df2['Date'].dt.year

In [ ]:
AED['Date'] = pd.to_datetime(AED['Date'])
AED['day'] = AED['Date'].dt.day
AED['month'] = AED['Date'].dt.month
AED['year'] = AED['Date'].dt.year

In [ ]:
result1 = pd.merge(df2, AED, left_on= ['day', 'month', 'year'], right_on = ['day', 'month', 'year'], how= 'left')

In [ ]:
result1['platform'] = 'Mumzworld'

In [ ]:
result1['Total USD']=result1['PO Amount']*result1['Close']
result1=result1.drop(['Close'], axis=1)
result1

result1["Country"] = "United Arab Emirates"

index_of_last_A = int(len(result1) * 0.6)

for i, row in result1.iterrows():
    if i <= index_of_last_A:
        result1.at[i, "Country"] = "Saudi Arabia"

result1=result1.drop(['Cost Price'], axis=1)

In [ ]:
result1= result1.merge(sku, how='left', left_on = "Vendor SKU" , right_on = "Product SKU")
# result1['Date'] = pd.to_datetime(result1['Date'])
cp['Date'] = pd.to_datetime(cp['Date'])
cp['Product'] = cp['Product'].apply(lambda x: x.title().lower())
result1['Product_Category'] = result1['Product_Category'].apply(lambda x: x.title().lower() if isinstance(x, str) else x)
# result1= pd.merge(result1, cp, how='left', left_on = ["Date", "Product_Category_2"] , right_on = ["Date", "Product"]) 

result1= pd.merge(result1, cp, how='left', left_on = ['Product_Category','day', 'month', 'year'] , right_on = ["Product", 'day', 'month', 'year'])
result1= pd.merge(result1, ff, how='left', left_on = "year" , right_on = "Year") 

In [ ]:
result1['Franchise fees'] = result1['Franchise fees'].str.replace('%', '').astype(float)
result1['Franchise fees_2'] = result1['Franchise fees'] / 100
result1['Franchise_amount'] = result1['Total USD'] * result1['Franchise fees_2']
df = pd.DataFrame(result1)
df['Total Cost'] = np.where(df['Cost Price'].isnull(), np.nan, df['Cost Price'] + df['Franchise_amount'])
df['Profit'] = np.where(df['Cost Price'].isnull(), np.nan, df['Total USD'] - df['Total Cost'])
df=df.drop(['Date','Date_y','UPC Code', 'Platform'], axis=1) 
df = df.rename(columns={'Date_x': 'Date', 'Franchise fees':'Franchise fees_percent', 'Franchise fees_2':'Franchise fees_2_percent'}) 

In [ ]:
df['Flag'] = False 
df.loc[df['Vendor SKU'].isin(['BL-Nintendo-3631', 'BL-Mermaid-3655', 'BL-Dino-3594', 'BL-Animals-3662', 'BL-Princess-3679', 'BL-Cars-3624', 'BL-Presents-3617', 'BL-Surfer-3600', 
                         'BL-Unicorn-3648', 'BL-Donuts-3686'],), 'Flag'] = True
df = df[df['Flag'] == False]
df = df.drop('Flag', axis=1) 

In [ ]:
gs = gc.open_by_url('https://docs.google.com/spreadsheets/d/1I8NdsQyAWgI7p237Hj7OsBou56IA1wYUHrXvuOFRj6M/edit#gid=67394343')
worksheet2 = gs.worksheet('Mumzworld')
worksheet2.clear()
set_with_dataframe(worksheet=worksheet2, dataframe=df, include_index=False,include_column_header=True, resize=True)